In [1]:
#import ollama 
import os
from tqdm import tqdm
import json
import signal
import argparse
import wandb

import sys

from  utilities import *

In [2]:
from PIL import Image
import matplotlib.pyplot as plt

### Testing for hateful memes

In [7]:

sys.argv = [
    'notebook',  
    '--modelname', 'llava:7b',
    '--prompt', 'Is this an offensive meme? Please answer with YES or NO. DO NOT mention the reason:',
    '--data', 'hateful_memes',
    '--data_path','/home1/pupil/goowfd/CVPR_2025/hateful_memes/',
    "--data_samples", 'train.jsonl',
    '--results_dir', '/home1/pupil/goowfd/CVPR_2025/hateful_memes/results/baselineExp',
    '--timeout', '20',
    '--model_unloading'
]



sys.argv = [
    'notebook',  
    '--modelname', 'llava:7b',
    '--prompt', 'Is this an offensive meme? Please answer with YES or NO. DO NOT mention the reason:',
    '--data', 'eurosat',
    '--data_path','/home1/pupil/rmf3mc/eurosat/2750/',
    "--data_samples", '../../data_split/split_zhou_EuroSAT.json',
    '--results_dir', '/home1/pupil/rmf3mc/eurosat/2750/results/baselineExp',
    '--timeout', '20',
    '--model_unloading'
]


In [8]:
parser = argparse.ArgumentParser(description="A script to run V-LLMs on different image classification datasets")

In [9]:
parser.add_argument("--modelname", type=str, required=True, help="The name of the V-LLM model")
parser.add_argument("--prompt", type=str, required=True, help="The prompt that you want to give to the V-LLM")
parser.add_argument("--data", type=str, required=True, help="Dataset name")
parser.add_argument("--data_path", type=str, required=True, help="Path to the image data dir")
parser.add_argument("--data_samples", type=str, required=True, help="Name of the samples to run on")
parser.add_argument("--results_dir", type=str, required=True, help="Folder name to save results")
parser.add_argument("--timeout", type=int, default=40, help="time out duration to skip one sample")
parser.add_argument("--model_unloading", action="store_true", help="Enables unloading mode. Every 100 sampels it unloades the model from the GPU to avoid carshing.")

args = parser.parse_args()

In [10]:
if args.data == 'hateful_memes':
    data =[]
    data_samples_path=os.path.join(args.data_path,args.data_samples)
    images_dir=os.path.join(args.data_path,'img')
    
    with open(data_samples_path, 'r') as file:
        for line in file: 
            record = json.loads(line)
            data.append({"path":os.path.join(images_dir,f'{record["img"][4:]}'),"label":record['label']})
            print(data[-1])
            
elif args.data == 'eurosat':
    data =[]
    data_samples_path=os.path.join(args.data_path,args.data_samples)
    print(data_samples_path)
    images_dir=os.path.join(args.data_path, args.data, '2750')
    print(images_dir)
    
    with open(data_samples_path, 'r') as file:
        data = json.load(file)
    
    
    print(data)
        


{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/42953.png', 'label': 0}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/23058.png', 'label': 0}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/13894.png', 'label': 0}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/37408.png', 'label': 0}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/82403.png', 'label': 0}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/16952.png', 'label': 0}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/76932.png', 'label': 0}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/70914.png', 'label': 0}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/02973.png', 'label': 0}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/58306.png', 'label': 0}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/79351.png', 'label': 1}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/34096.png', 'label': 0}
{'path': '/home1

{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/60314.png', 'label': 0}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/05327.png', 'label': 0}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/56248.png', 'label': 1}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/53219.png', 'label': 0}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/42183.png', 'label': 0}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/26950.png', 'label': 1}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/24986.png', 'label': 0}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/89326.png', 'label': 0}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/52964.png', 'label': 0}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/17352.png', 'label': 1}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/82693.png', 'label': 1}
{'path': '/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/59143.png', 'label': 0}
{'path': '/home1

### Testing for eurosat

In [7]:
model_name = args.modelname
results_dir=os.path.join(args.results_dir)
os.makedirs(results_dir, exist_ok=True)
results_file_name=os.path.join(results_dir,f"{model_name}.json")

In [8]:
ollama.pull(model_name)

timeout_duration = args.timeout

options= {  # new
            "seed": 123,
            "temperature": 0,
            "num_ctx": 2048, # must be set, otherwise slightly random output
        }

model_labels = {}
prompt = args.prompt
count = 0

In [10]:
for image_details in tqdm(data):
    count = count + 1
    image_path = image_details['path']


    
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(timeout_duration)  

    try:
        if args.model_unloading and count % 99 == 0:
            response = ollama.generate(model=model_name, prompt=prompt, images=[image_path], options=options, keep_alive=0)
        else:
            response = ollama.generate(model=model_name, prompt=prompt, images=[image_path], options=options)
    
    except TimeoutException:
        print(f"Prompt for {image_name} took longer than {timeout_duration} seconds. Moving to the next one.")
        label = None
        
    finally:
        signal.alarm(0)  
    
    label = check_yes_no(response['response'])
    model_labels[image_path] = label

    
    

  0%|▍                                                                                                         | 35/8500 [00:25<1:42:05,  1.38it/s]


KeyboardInterrupt: 

In [ ]:
with open(results_file_name, 'w') as fp:
    json.dump(model_labels, fp)

In [11]:
results_file_name

'/home1/pupil/goowfd/CVPR_2025/hateful_memes/results/baselineExp/llava:7b.json'